In [17]:
import pydae.build_cffi as db
from pydae.bmapu import bmapu_builder
from pydae.bmapu.syns.syns import add_syns
from pydae.bmapu.vsgs.vsgs import add_vsgs
from pydae.bmapu.genapes.genapes import add_genapes

from pydae.grid_bpu import bpu

import pydae.build_cffi as db
import sympy as sym
import json        

data = {
    "sys":{"name":"sys2buses","S_base":100e6, "K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.0},       
    "buses":[{"name":"1", "P_W":0.0,"Q_var":0.0,"U_kV":20.0},
            {"name":"2", "P_W":0.0,"Q_var":0.0,"U_kV":20.0}],
    "lines":[{"bus_j":"1", "bus_k":"2", "X_pu":0.15,"R_pu":0.0, "S_mva":900.0}],
    "syns":[
       {"bus":"1","S_n":100e6,
        "X_d":1.8,"X1d":0.3, "T1d0":8.0,    
        "X_q":1.7,"X1q":0.55,"T1q0":0.4,  
        "R_a":0.0025,"X_l": 0.2, 
        "H":6.175,"D":1.0,
        "Omega_b":314.1592653589793,"omega_s":1.0,"K_sec":0.0,
        #"avr":{"type":"sexs","K_a":300, "T_r":0.02, 'K_ai':1e-6,"V_min":-10000.0,"V_max":5.0, "v_pss":0.0, "v_ref":1.03,"K_aw":10},
        #"gov":{"type":"agov1","Droop":0.05,"T_1":1.0,"T_2":2.0,"T_3":10.0, "p_c":700/900,"omega_ref":1.0, "K_imw":0.0},
        #"pss":{"type":"pss_kundur","T_wo":10.0,"T_1":0.1, 'T_2':0.1, 'K_stab':1.0},
        "K_delta":0.001},
       #{"bus":"2","S_n":100e6,
       # "X_d":1.8,"X1d":0.3, "T1d0":8.0,    
       # "X_q":1.7,"X1q":0.55,"T1q0":0.4,  
       # "R_a":0.0025,"X_l": 0.2, 
       # "H":6.175,"D":1.0,
       # "Omega_b":314.1592653589793,"omega_s":1.0,"K_sec":0.0,
       # "avr":{"type":"sexs","K_a":300, "T_r":0.02, 'K_ai':1e-6,"V_min":-10000.0,"V_max":5.0, "v_pss":0.0, "v_ref":1.03,"K_aw":10},
       # "gov":{"type":"agov1","Droop":0.05,"T_1":1.0,"T_2":2.0,"T_3":10.0, "p_c":700/900,"omega_ref":1.0, "K_imw":0.0},
       # "pss":{"type":"pss_kundur","T_wo":10.0,"T_1":0.1, 'T_2':0.1, 'K_stab':1.0},
       # "K_delta":0.0}
    ],
    #"vsgs":[
    #    {"bus":"1","type":"vsg_ll",'S_n':10e6,'F_n':50,'K_delta':0.0,
    #    'R_v':0.01,'X_v':0.1,'K_p':1.0,'K_i':0.1,'K_g':0.0,'K_q':20.0,
    #    'T_q':0.1,'K_p_v':1e-6,'K_i_v':1e-6}],
    "genapes":[{"bus":"2","S_n":1000e6,"F_n":50.0,"R_v":0.0,"X_v":0.1,"K_delta":0.001,"K_alpha":0.001}],
    }

grid = bmapu_builder.bmapu(data)
add_syns(grid)
#add_vsgs(grid)
add_genapes(grid)

#add_vsgs(grid)
omega_coi = sym.Symbol("omega_coi", real=True)  

grid.dae['g'] += [ -omega_coi + grid.omega_coi_numerator/grid.omega_coi_denominator]
grid.dae['y_ini'] += [ omega_coi]
grid.dae['y_run'] += [ omega_coi]

# secondary frequency control
xi_freq = sym.Symbol("xi_freq", real=True) 
p_agc = sym.Symbol("p_agc", real=True)  
K_p_agc = sym.Symbol("K_p_agc", real=True) 
K_i_agc = sym.Symbol("K_i_agc", real=True) 
K_xif  = sym.Symbol("K_xif", real=True)

epsilon_freq = 1-omega_coi
g_agc = [ -p_agc + K_p_agc*epsilon_freq + K_i_agc*xi_freq ]
y_agc = [  p_agc]
x_agc = [ xi_freq]
f_agc = [epsilon_freq - K_xif*xi_freq]

grid.dae['g'] += g_agc
grid.dae['y_ini'] += y_agc
grid.dae['y_run'] += y_agc
grid.dae['f'] += f_agc
grid.dae['x'] += x_agc
grid.dae['params_dict'].update({'K_p_agc':grid.sys['K_p_agc'],'K_i_agc':grid.sys['K_i_agc']})

if 'K_xif' in grid.sys:
    grid.dae['params_dict'].update({'K_xif':grid.sys['K_xif']})
else:
    grid.dae['params_dict'].update({'K_xif':0.0})
            
grid2 = bpu(data_input=data)


In [18]:
for item in ['f','g','x','y_ini','y_run','u_ini','u_run']:
    for item1,item2 in zip(grid.dae['y_ini'],grid2.dae['y_ini']):
        if not (item1==item2):
            print(item1)
            print(item2)

omega_2_0
v_f_1
i_d_2_0
omega_2_0
i_q_2_0
i_d_2_0
p_s_2_0
i_q_2_0
q_s_2_0
p_s_2_0
omega_coi
q_s_2_0
p_agc
omega_coi
omega_2_0
v_f_1
i_d_2_0
omega_2_0
i_q_2_0
i_d_2_0
p_s_2_0
i_q_2_0
q_s_2_0
p_s_2_0
omega_coi
q_s_2_0
p_agc
omega_coi
omega_2_0
v_f_1
i_d_2_0
omega_2_0
i_q_2_0
i_d_2_0
p_s_2_0
i_q_2_0
q_s_2_0
p_s_2_0
omega_coi
q_s_2_0
p_agc
omega_coi
omega_2_0
v_f_1
i_d_2_0
omega_2_0
i_q_2_0
i_d_2_0
p_s_2_0
i_q_2_0
q_s_2_0
p_s_2_0
omega_coi
q_s_2_0
p_agc
omega_coi
omega_2_0
v_f_1
i_d_2_0
omega_2_0
i_q_2_0
i_d_2_0
p_s_2_0
i_q_2_0
q_s_2_0
p_s_2_0
omega_coi
q_s_2_0
p_agc
omega_coi
omega_2_0
v_f_1
i_d_2_0
omega_2_0
i_q_2_0
i_d_2_0
p_s_2_0
i_q_2_0
q_s_2_0
p_s_2_0
omega_coi
q_s_2_0
p_agc
omega_coi
omega_2_0
v_f_1
i_d_2_0
omega_2_0
i_q_2_0
i_d_2_0
p_s_2_0
i_q_2_0
q_s_2_0
p_s_2_0
omega_coi
q_s_2_0
p_agc
omega_coi


In [19]:
sys_dict = {'name':'prueba',
    'params_dict':grid.dae['params_dict'],
    'f_list':grid.dae['f'],
    'g_list':grid.dae['g'],
    'x_list':grid.dae['x'],
    'y_ini_list':grid.dae['y_ini'],
    'y_run_list':grid.dae['y_run'],
    'u_run_dict':grid.dae['u_run_dict'],
    'u_ini_dict':grid.dae['u_ini_dict'],
    'h_dict':grid.dae['h_dict']}

bldr = db.builder(sys_dict, verbose=True)
bldr.build()

check_system (time: 0.0)
computing jacobians Fx_run,Fy_run  (time: 0.015 s)
computing jacobians Gx_run,Gy_run  (time: 0.036 s)
computing jacobians Fu_run,Gu_run  (time: 0.083 s)
computing jacobians Fx_ini,Fy_ini  (time: 0.107 s)
computing jacobians Gx_ini,Gy_ini  (time: 0.117 s)
computing jacobians Hx_run,Hy_run,Hu_run  (time: 0.15703725814819336 s)
end system  (time: 0.180 s)
computing jac_ini (time: 0.0)
computing jac_run (time: 0.0009999275207519531)
computing jac_trap (time: 0.002000570297241211)
end of jacobians computation (time: 0.003)
writting f_ini and g_ini code (time: 0.000 s)
writting f_run and g_run code (time: 0.058 s)
writting h_run code (time: 0.117 s)
converting jac_ini to sp_jac_ini  (time: 0.137 s)
running sym2rhs for sp_jac_ini (time: 0.137 s)
converting jac_run to sp_jac_run  (time: 0.263 s)
running sym2rhs for sp_jac_run (time: 0.264 s)
converting jac_trap to sp_jac_trap  (time: 0.394 s)
running sym2rhs for sp_jac_trap (time: 0.395 s)
wrtting  de_jac_trap code (ti

VerificationError: LinkError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\VC\\Tools\\MSVC\\14.29.30133\\bin\\HostX86\\x64\\link.exe' failed with exit code 1104

In [9]:
with open('xy_0_2.json','w') as fobj:
    fobj.write(json.dumps(grid.dae['xy_0_dict'],indent=4))

In [10]:
import prueba 
model = prueba.model()
model.report_y()

V_1   =  0.00
theta_1 =  0.00
V_2   =  0.00
theta_2 =  0.00
i_d_ref_1 =  0.00
i_q_ref_1 =  0.00
p_1   =  0.00
q_1   =  0.00
e_qv_1 =  0.00
omega_2_0 =  0.00
i_d_2_0 =  0.00
i_q_2_0 =  0.00
p_s_2_0 =  0.00
q_s_2_0 =  0.00
omega_coi =  0.00
p_agc =  0.00


In [11]:
model.ini({'p_ref_1':0.0,"K_xif":0.1},'xy_0_2.json')
model.report_params()

S_base =100000000.00
g_1_2 = 0.00
b_1_2 =-60.00
bs_1_2 = 0.00
U_1_n =20000.00
U_2_n =20000.00
S_n_1 =10000000.00
F_n_1 =50.00
K_delta_1 = 0.00
K_p_1 = 1.00
K_i_1 = 0.10
K_g_1 = 0.00
R_v_1 = 0.01
X_v_1 = 0.10
K_q_1 =20.00
T_q_1 = 0.10
K_p_v_1 = 0.00
K_i_v_1 = 0.00
S_n_2_0 =1000000000.00
F_n_2_0 =50.00
X_v_2_0 = 0.10
R_v_2_0 = 0.00
K_delta_2_0 = 0.00
K_alpha_2_0 = 0.00
K_p_agc = 0.00
K_i_agc = 0.00
K_xif = 0.10


In [12]:
model.report_params()

S_base =100000000.00
g_1_2 = 0.00
b_1_2 =-60.00
bs_1_2 = 0.00
U_1_n =20000.00
U_2_n =20000.00
S_n_1 =10000000.00
F_n_1 =50.00
K_delta_1 = 0.00
K_p_1 = 1.00
K_i_1 = 0.10
K_g_1 = 0.00
R_v_1 = 0.01
X_v_1 = 0.10
K_q_1 =20.00
T_q_1 = 0.10
K_p_v_1 = 0.00
K_i_v_1 = 0.00
S_n_2_0 =1000000000.00
F_n_2_0 =50.00
X_v_2_0 = 0.10
R_v_2_0 = 0.00
K_delta_2_0 = 0.00
K_alpha_2_0 = 0.00
K_p_agc = 0.00
K_i_agc = 0.00
K_xif = 0.10
